In [1]:
from flask import Flask, render_template, request, jsonify
import pandas as pd
import plotly.express as px
import plotly.io as pio
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import base64
from io import BytesIO
import json
from datetime import datetime, timedelta
import os
import hashlib
import numpy as np

app = Flask(__name__)

from pymongo import MongoClient

In [14]:
client = MongoClient("mongodb://localhost:27017/")
db = client["Dashboard"]
collection = db["Coffee"]

In [15]:
df = pd.DataFrame(list(collection.find()))

# Drop MongoDB's internal _id column
if '_id' in df.columns:
    df.drop('_id', axis=1, inplace=True)

In [16]:
def preprocess_dataframe(df):
    df['date'] = pd.to_datetime(df['transaction_date'] + '-' + df['transaction_time'], format="mixed")
    df.drop(['transaction_date', 'transaction_time'], axis=1, inplace=True)
    df.set_index('date', inplace=True)
    df['month'] = df.index.month
    df['is_weekend'] = df.index.weekday >= 5
    df['revenue'] = df['transaction_qty'] * df['unit_price']
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['day_name'] = df.index.day_name()
    return df

df = preprocess_dataframe(df)

In [17]:
print(df.columns.tolist())

['transaction_id', 'transaction_qty', 'store_id', 'store_location', 'product_id', 'unit_price', 'product_category', 'product_type', 'product_detail', 'month', 'is_weekend', 'revenue', 'hour', 'day', 'day_name']
